In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score,GridSearchCV


In [45]:
## Importing the data
columns=['Sex','Length','Diameter','Height','Whole Weight','Shucked Weight','Viscera Weight','Shell Weight','Rings']
df=pd.read_csv('abalone.data',names=columns)

In [46]:
## Checking first few rows
df.head()

,Sex,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [47]:
# Checking the information
## We find no null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   object 
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole Weight    4177 non-null   float64
 5   Shucked Weight  4177 non-null   float64
 6   Viscera Weight  4177 non-null   float64
 7   Shell Weight    4177 non-null   float64
 8   Rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


In [48]:
# Make a copy of the data
df_copy=df.copy()


In [49]:
# Splitting into features and labels
X=df_copy.drop('Rings',axis=1)
y=df_copy['Rings']

In [50]:
## Checking y series

In [51]:
print(y[:5])

0    15
1     7
2     9
3    10
4     7
Name: Rings, dtype: int64


In [52]:
#Checking the shape
print(X.shape)
print(y.shape)

(4177, 8)
(4177,)


In [53]:
X_transformed=pd.get_dummies(X,prefix='Sex',drop_first=True)

In [54]:
X_transformed

,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,1,0
...,...,...,...,...,...,...,...,...,...
4172,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,0,0
4173,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,0,1
4174,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,0,1
4175,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,0,0


In [55]:
X_transformed.columns=['Length','Diameter','Height','Whole Weight','Shucked Weight','Viscera Weight','Shell Weight','Is_Infant','Is_Male']

In [56]:
X_transformed

,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Is_Infant,Is_Male
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,1,0
...,...,...,...,...,...,...,...,...,...
4172,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,0,0
4173,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,0,1
4174,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,0,1
4175,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,0,0


In [57]:
#Lets Scale the data on common scale
scaler=StandardScaler()
numeric_cols=list(X_transformed.iloc[:,0:7].columns)
numeric_cols
transformer=ColumnTransformer([('scaler',StandardScaler(),numeric_cols)],remainder='passthrough')
scaled_transformed_X=transformer.fit_transform(X_transformed)
scaled_transformed_X_df=pd.DataFrame(scaled_transformed_X,columns=X_transformed.columns)

In [58]:
scaled_transformed_X_df.head()

,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Is_Infant,Is_Male
0,-0.574558,-0.432149,-1.064424,-0.641898,-0.607685,-0.726212,-0.638217,0.0,1.0
1,-1.448986,-1.439929,-1.183978,-1.230277,-1.170910,-1.205221,-1.212987,0.0,1.0
2,0.050033,0.122130,-0.107991,-0.309469,-0.463500,-0.356690,-0.207139,0.0,0.0
3,-0.699476,-0.432149,-0.347099,-0.637819,-0.648238,-0.607600,-0.602294,0.0,1.0
4,-1.615544,-1.540707,-1.423087,-1.272086,-1.215968,-1.287337,-1.320757,1.0,0.0


In [59]:
# Splitting into training,validation and test sets
X_train,X_test,y_train,y_test=train_test_split(scaled_transformed_X_df,y,test_size=0.2,random_state=42)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=42)

In [60]:
print(len(X_train),len(X_test),len(X_val),len(y_train),len(y_test),len(y_val))

2672 836 669 2672 836 669


In [61]:
print(len(X_train)/len(X_transformed)*100)
print(len(X_test)/len(X_transformed)*100)
print(len(X_val)/len(X_transformed)*100)

63.96935599712712
20.01436437634666
16.016279626526213


In [62]:
models={'LR':LinearRegression(),'GBR':GradientBoostingRegressor(),'RF':RandomForestRegressor(),'CART':DecisionTreeRegressor(),'SVR':SVR()}

In [63]:
for names,model in models.items():
    clf=model.fit(X_train,y_train)
    score=cross_val_score(clf,X_train,y_train,scoring='neg_mean_squared_error',cv=5)
    print("**********Results**************")
    print("Model Name",names)
    print("Model Score",score)
    print("Model Mean",score.mean())
    print("Model Std",score.std())
    
    
    
    
    

**********Results**************
Model Name LR
Model Score [-4.993395   -5.4711318  -4.75267429 -4.09861414 -4.76247795]
Model Mean -4.815658635129559
Model Std 0.44321175654123124
**********Results**************
Model Name GBR
Model Score [-4.41198191 -5.73954151 -4.79023555 -4.04408068 -4.49943049]
Model Mean -4.697054027846203
Model Std 0.5730895765096196
**********Results**************
Model Name RF
Model Score [-4.544      -5.66793121 -4.79384326 -4.24594719 -4.59811461]
Model Mean -4.769967254226609
Model Std 0.4820997609256954
**********Results**************
Model Name CART
Model Score [ -7.72336449 -10.96448598  -9.37453184  -8.85018727  -8.93820225]
Model Mean -9.170154363120867
Model Std 1.0498359665294337
**********Results**************
Model Name SVR
Model Score [-4.24595501 -5.66962676 -4.92697686 -3.94525147 -4.77320201]
Model Mean -4.712202422779621
Model Std 0.5953336518990079


In [64]:
#Wow Random Forest Regressor has performed quite well in terms of Standard Deviation
#So we will select Random Forest has our baseline model

In [65]:
#Time to Tune our hyperparameters using GridCVDSearch
param_grid=[{'n_estimators':[1,5,10,20,50],'max_features':[5,10,20],'max_depth':[10,50,None],'bootstrap':[True,False]}]
grid=GridSearchCV(RandomForestRegressor(),param_grid=param_grid,cv=10,scoring='neg_mean_squared_error')

In [66]:
grid.fit(X_val,y_val)

C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 1029, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 847, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 765, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 1029, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 847, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 765, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 1029, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 847, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 765, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 1029, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 847, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 765, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 1029, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 847, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 765, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

...

C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 1029, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 847, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 765, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 1029, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 847, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 765, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 1029, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 847, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 765, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\manis\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 1029, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 847, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\manis\anaconda3\lib\site-packages\joblib\parallel.py", line 765, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid=[{'bootstrap': [True, False],
                          'max_depth': [10, 50, None],
                          'max_features': [5, 10, 20],
                          'n_estimators': [1, 5, 10, 20, 50]}],
             scoring='neg_mean_squared_error')

In [67]:
cvres=grid.cv_results_
for mean_score,params in zip(cvres['mean_test_score'],cvres['params']):
    print(np.sqrt(-mean_score),params)

2.863561453763744 {'bootstrap': True, 'max_depth': 10, 'max_features': 5, 'n_estimators': 1}
2.329583324768541 {'bootstrap': True, 'max_depth': 10, 'max_features': 5, 'n_estimators': 5}
2.230773905228679 {'bootstrap': True, 'max_depth': 10, 'max_features': 5, 'n_estimators': 10}
2.175026451613495 {'bootstrap': True, 'max_depth': 10, 'max_features': 5, 'n_estimators': 20}
2.1359892523988426 {'bootstrap': True, 'max_depth': 10, 'max_features': 5, 'n_estimators': 50}
nan {'bootstrap': True, 'max_depth': 10, 'max_features': 10, 'n_estimators': 1}
nan {'bootstrap': True, 'max_depth': 10, 'max_features': 10, 'n_estimators': 5}
nan {'bootstrap': True, 'max_depth': 10, 'max_features': 10, 'n_estimators': 10}
nan {'bootstrap': True, 'max_depth': 10, 'max_features': 10, 'n_estimators': 20}
nan {'bootstrap': True, 'max_depth': 10, 'max_features': 10, 'n_estimators': 50}
nan {'bootstrap': True, 'max_depth': 10, 'max_features': 20, 'n_estimators': 1}
nan {'bootstrap': True, 'max_depth': 10, 'max_fe

In [68]:
grid_best_estimator=grid.best_estimator_

In [69]:
#Evaluating the metric
y_preds=grid_best_estimator.predict(X_test)


In [70]:
y_preds

array([10.19741904,  9.81061034, 15.15412225, 11.76119431, 12.04714386,
       12.19488478,  9.0704955 ,  8.92582683,  7.26200425,  9.98528501,
       10.87775537,  7.78166667, 12.69645249,  7.09813606,  7.94755696,
       12.46230707, 14.02384367,  9.99319222, 11.67486529,  7.64340628,
       14.13835924, 10.57354543, 11.71922619, 10.22356795,  8.98143436,
        7.61881813,  9.46790362,  7.37183052,  9.83567214, 10.42461108,
       10.8699642 , 10.38142698,  6.65798715, 10.09171762,  9.22471861,
        9.67126472,  5.35690867,  4.16      ,  8.30952381,  6.70663977,
       11.36762779, 13.04827392, 10.1993975 ,  9.16443467, 11.32644931,
       10.81694083,  9.65009313,  9.55381334, 12.66330758,  5.26      ,
        6.18962814, 15.89333333,  9.46094624,  4.36      ,  6.25669818,
       10.90338761, 10.02301558, 12.78195042,  9.85682203,  8.09333333,
       11.02483834, 12.3064446 ,  9.48598656,  7.2229981 ,  7.47857753,
       11.40345674,  4.48      ,  6.51603363,  9.23857962, 11.72

In [71]:
errors=abs(y_preds-y_test)

In [72]:
mape = np.mean(100 * (errors / y_test))

In [73]:
accuracy=100-mape

In [74]:
print('The best model has an accuracy of',round(accuracy,2),'%')

The best model has an accuracy of 84.69 %


In [75]:
#RMSE
grid_mse=mean_squared_error(y_test,y_preds)
rmse=np.sqrt(grid_mse)
print('The best model has a RMSE of',round(rmse,2))

The best model has a RMSE of 2.32


In [76]:
y_test_df=pd.Series(y_test)

In [77]:
y_test_df.reset_index(drop=True,inplace=True)

In [78]:
y_test=pd.Series(y_test_df)

In [79]:
y_test

0       9
1       8
2      16
3       9
4      14
       ..
831    11
832     8
833    10
834     8
835     7
Name: Rings, Length: 836, dtype: int64

In [88]:
y_pred_series=pd.Series(y_preds)

In [93]:
predicted_values=pd.DataFrame(y_pred_series,columns=['Predicted Values'])

In [94]:
predicted_values

,Predicted Values
0,10.197419
1,9.810610
2,15.154122
3,11.761194
4,12.047144
...,...
831,10.725827
832,9.598157
833,10.232830
834,8.732145


In [103]:
actual_values=pd.DataFrame(y_test)

In [104]:
actual_values

,Rings
0,9
1,8
2,16
3,9
4,14
...,...
831,11
832,8
833,10
834,8


In [105]:
actual_values

,Rings
0,9
1,8
2,16
3,9
4,14
...,...
831,11
832,8
833,10
834,8


In [109]:
actual_predicted_values=pd.concat([actual_values,predicted_values],axis=1)

In [110]:
actual_predicted_values

,Rings,Predicted Values
0,9,10.197419
1,8,9.810610
2,16,15.154122
3,9,11.761194
4,14,12.047144
...,...,...
831,11,10.725827
832,8,9.598157
833,10,10.232830
834,8,8.732145


In [111]:
print(type(actual_predicted_values))

<class 'pandas.core.frame.DataFrame'>


In [113]:
actual_predicted_values.columns=['Actual Values','Predicted Values']

In [114]:
actual_predicted_values

,Actual Values,Predicted Values
0,9,10.197419
1,8,9.810610
2,16,15.154122
3,9,11.761194
4,14,12.047144
...,...,...
831,11,10.725827
832,8,9.598157
833,10,10.232830
834,8,8.732145
